In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras.applications.efficientnet as efn

EFNETS = (efn.EfficientNetB0, efn.EfficientNetB1, efn.EfficientNetB2, efn.EfficientNetB3, 
          efn.EfficientNetB4, efn.EfficientNetB5, efn.EfficientNetB6, efn.EfficientNetB7)
    
print(f'tf version: {tf.__version__}')

tf version: 2.3.1


In [2]:
N_CLASSES = 9
SEED = 17
IMAGES_DIR = '/kaggle/input/skillbox-emotions/'
TEST_DIR = IMAGES_DIR + '/test_kaggle'

train_df = pd.read_csv('/kaggle/input/skillbox-computer-vision-project/train.csv').iloc[:, 1:]
sub = pd.read_csv('/kaggle/input/skillbox-computer-vision-project/sample_submission.csv')

train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=SEED)

In [3]:
train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
                                    horizontal_flip=True, rotation_range=15, 
                                    width_shift_range=0.15, height_shift_range=0.15)
val_data_gen = tf.keras.preprocessing.image.ImageDataGenerator()
test_data_gen = tf.keras.preprocessing.image.ImageDataGenerator()

In [4]:
EFNET_NO = 1
BATCH_SIZE = 64
IMSIZES = (224, 240, 260, 300, 380, 456, 528, 600)
IMAGE_SIZE = IMSIZES[EFNET_NO]

train_data = train_data_gen.flow_from_dataframe(
              train_df, directory=IMAGES_DIR, x_col='image_path', y_col='emotion', class_mode='sparse',
              target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE, shuffle=True
          )
val_data = val_data_gen.flow_from_dataframe(
              val_df, directory=IMAGES_DIR, x_col='image_path', y_col='emotion', class_mode='sparse',
              target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE, shuffle=False
          )
test_data = test_data_gen.flow_from_dataframe(
              sub, directory=TEST_DIR, x_col='image_path', y_col=None, class_mode=None,
              target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE, shuffle=False
          )

Found 45041 validated image filenames belonging to 9 classes.
Found 5005 validated image filenames belonging to 9 classes.
Found 5000 validated image filenames.


In [5]:
MAPPING = {k: i for i, k in train_data.class_indices.items()}

In [6]:
class MySequential(tf.keras.Sequential):
    def predict_tta(self, data_generator):
        tta_preds = []
        i = 0
        images_count = data_generator.n
        for curr_im_batch in data_generator:
            for curr_img in curr_im_batch:
                pred = self.predict(curr_img[None, ...])
                pred_f = self.predict(np.flip(curr_img, axis=1)[None, ...])
                tta_pred = np.stack((pred, pred_f)).mean(axis=0)[0]
                tta_preds.append(tta_pred)
                
                i += 1
                if i % 1000 == 0:
                    print(f'{i}/{images_count} images processed')
                
            if i == images_count:
                print('Done')
                break
            
        return np.array(tta_preds)

In [7]:
model = MySequential([
    EFNETS[EFNET_NO](
        input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
        weights='imagenet',
        include_top=False),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(N_CLASSES, activation='softmax')
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='sparse_categorical_crossentropy',
    metrics='accuracy')
model.summary()

27025408/27018416 [==============================] - 1s 0us/step
Model: "my_sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb1 (Functional)  (None, 8, 8, 1280)        6575239   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 9)                 11529     
Total params: 6,586,768
Trainable params: 6,524,713
Non-trainable params: 62,055
_________________________________________________________________


In [8]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'model.h5', save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_accuracy", patience=3, min_lr=3e-7, mode='max')

In [9]:
EPOCHS = 20
history = model.fit(
    train_data, validation_data=val_data, 
    epochs=EPOCHS, callbacks=[checkpoint, lr_reducer])

Epoch 1/20
704/704 [==============================] - ETA: 0s - loss: 1.5095 - accuracy: 0.4495
Epoch 00001: val_accuracy improved from -inf to 0.49151, saving model to model.h5
704/704 [==============================] - 1329s 2s/step - loss: 1.5095 - accuracy: 0.4495 - val_loss: 1.4278 - val_accuracy: 0.4915
Epoch 2/20
704/704 [==============================] - ETA: 0s - loss: 1.3295 - accuracy: 0.5166
Epoch 00002: val_accuracy improved from 0.49151 to 0.49670, saving model to model.h5
704/704 [==============================] - 1085s 2s/step - loss: 1.3295 - accuracy: 0.5166 - val_loss: 1.3746 - val_accuracy: 0.4967
Epoch 3/20
704/704 [==============================] - ETA: 0s - loss: 1.2677 - accuracy: 0.5377
Epoch 00003: val_accuracy did not improve from 0.49670
704/704 [==============================] - 1086s 2s/step - loss: 1.2677 - accuracy: 0.5377 - val_loss: 1.4466 - val_accuracy: 0.4845
Epoch 4/20
704/704 [==============================] - ETA: 0s - loss: 1.2194 - accuracy: 0.

In [10]:
model.load_weights('model.h5')

In [11]:
tta_preds = model.predict_tta(test_data)

1000/5000 images processed
2000/5000 images processed
3000/5000 images processed
4000/5000 images processed
5000/5000 images processed
Done


In [12]:
def make_submission(preds, submission_df=sub, out_file='submission.csv', class_names=MAPPING):
    preds = preds.argmax(axis=-1)
    preds = [class_names[i] for i in preds]
    submission_df['emotion'] = preds
    submission_df.to_csv(out_file, index=False)
    submission_df.head()

In [13]:
make_submission(preds=tta_preds, out_file='tta_preds.csv')